In [1]:
import numpy as np
import pandas as pd
import evaluate
import torch
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss

from preprocessing import climateBUG_to_reduced_climateBUG



In [2]:
# This should always output true now, but worth checking
print(f"is GPU available: {torch.cuda.is_available()}")

is GPU available: True


In [4]:
# The climate-fever dataset is in a format which the transformers Trainer does not understand
# It must be preprocessed using the functions in preprocessing.py

df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-training-dataset.json")
preprocessed_training_df = climateBUG_to_reduced_climateBUG(df)

df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-testing-dataset.json")
preprocessed_testing_df = climateBUG_to_reduced_climateBUG(df)


                                                statement  label
0       2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REP...      0
1       Retail     banking     achieved     a     stro...      0
2       Crédit   Agricole   Assurances   captured  fur...      0
3       Our  business  is  also  exposed  to  the  ups...      0
4       Crédit  Agricole  assumed  its   responsibilit...      0
...                                                   ...    ...
121061  -0132 filed with the AMF on 7 March 2017; The ...      0
121062  Page ■Description of the main risks and contin...      1
121063  Page  ■Information on payment terms (L.441-6-1...      0
121064  Page ■Information on the labour and environmen...      0
121065       2004 for an environmental management system.      1

[121066 rows x 2 columns]
                                               statement  label
0      4CRÉDIT AGRICOLE S.A. – 2020-2021 INTEGRATED R...      0
1      p.11Exceptional financial  strength .............      0
2

In [10]:
training_dataset = Dataset.from_pandas(preprocessed_training_df)
testing_dataset = Dataset.from_pandas(preprocessed_testing_df)
training_dataset[0]

{'statement': '2015REGISTRATION DOCUMENT ANNUAL FINANCIAL REPORT2 CRÉDIT AGRICOLE S.A.      2015 REGISTRATION DOCUMENT MESSAGE FROM THE CHAIRMAN AND THE CHIEF EXECUTIVE OFFICER MESSAGE FROM THE CHAIRMANAND THE CHIEF EXECUTIVE OFFICERPHILIPPE BRASSAC, CHIEF EXECUTIVE OFFICER AND DOMINIQUE LEFEBVRE, CHAIRMAN OF CRÉDIT AGRICOLE S.A.A robust Group delivering sustainable performanceLeading  a  bank  with  a  EUR  698  billion   loan   book(1)',
 'label': 0}

In [ ]:
training_dataset = training_dataset.shuffle(seed=13)
testing_dataset = testing_dataset.shuffle(seed=13)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# This splits our dataset so that we use 90% of it for training, and 10% for testing
split_dataset = dataset.train_test_split(test_size=0.1)

In [9]:
def custom_tokenize(examples):
    # The code block below this one can be used to find what the max_length should be set to.
    # Otherwise you have too much padding
    # Consider this properly later
    tokenized_output = tokenizer(
        text=[f"Claim: {claim} Evidence: {evidence}" for claim, evidence in zip(examples["claim"], examples["evidence"])],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_training_dataset = split_dataset["train"].map(custom_tokenize, batched=True)
tokenized_testing_dataset = split_dataset["test"].map(custom_tokenize, batched=True)

Map:   0%|          | 0/6907 [00:00<?, ? examples/s]

Map:   0%|          | 0/768 [00:00<?, ? examples/s]

In [10]:

# Just for viewing purposes. Input_ids are the tokens, and attention_masks are whether they represent actual words or not.
# The max_length is set to 512 so every entry has been padded to be this long, which may be unnecessary
print(tokenized_training_dataset[6]["claim"])
print(tokenized_training_dataset[6]["evidence"])
print(tokenized_training_dataset[6]["labels"])
print(tokenized_training_dataset[6]["input_ids"])
print(tokenized_training_dataset[6]["attention_mask"])

print(tokenized_testing_dataset[2]["claim"])
print(tokenized_testing_dataset[2]["evidence"])
print(tokenized_training_dataset[2]["labels"])
print(tokenized_testing_dataset[2]["input_ids"])
print(tokenized_testing_dataset[2]["attention_mask"])

The Business Council, the Minerals Council, the Australian Industry Group, the Australian Chamber of Commerce and Industry, have all called for the [carbon] tax to be repealed.
On 1 July 2012, the Australian Federal government introduced a carbon price of AUD$23 per tonne of emitted CO2-e on selected fossil fuels consumed by major industrial emitters and government bodies such as councils.
1
[0, 45699, 35, 20, 2090, 1080, 6, 5, 20710, 1536, 1080, 6, 5, 2059, 5354, 826, 6, 5, 2059, 7514, 9, 5669, 8, 5354, 6, 33, 70, 373, 13, 5, 646, 23612, 742, 629, 7, 28, 29643, 4, 27956, 35, 374, 112, 550, 1125, 6, 5, 2059, 1853, 168, 2942, 10, 4363, 425, 9, 17551, 1629, 1922, 228, 4866, 858, 9, 37141, 1437, 50265, 12, 242, 15, 3919, 1437, 50348, 1437, 50424, 13056, 30, 538, 2683, 29901, 2696, 8, 168, 3738, 215, 25, 14751, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
# Remove ignore_mismatched_sizes when needed - this replaces the head of the pretrained model (because if using
# climateBERT/environmental-claims, it has already been fine tuned and has 2 labels
model = AutoModelForSequenceClassification.from_pretrained(
    "climatebert/distilroberta-base-climate-f",
    num_labels=3,
    # ignore_mismatched_sizes=True
).to("cuda")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(next(model.parameters()).device)

cuda:0


In [13]:
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [14]:
# This is where we set the hyperparameters
training_args = TrainingArguments(
    output_dir="./results/climateBERT-base/climate_fever/first_run",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="no",
    save_steps=500,
    fp16=True,                          # Use 16-bit floating point instead of 32 - makes computation faster
    warmup_ratio=0.05,                    # Allows the model to adapt a little
    # gradient_accumulation_steps=2       # Might help with OOM errors, if we have them
    learning_rate=3e-5,
    push_to_hub=False,
)

In [15]:
trainer  = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [16]:
trainer.train()

  0%|          | 0/2592 [00:00<?, ?it/s]

{'loss': 1.0774, 'grad_norm': 3.270073890686035, 'learning_rate': 1.153846153846154e-05, 'epoch': 0.06}
{'loss': 0.9022, 'grad_norm': 4.453432083129883, 'learning_rate': 2.2846153846153845e-05, 'epoch': 0.12}
{'loss': 0.8809, 'grad_norm': 8.023266792297363, 'learning_rate': 2.976848090982941e-05, 'epoch': 0.17}
{'loss': 0.8858, 'grad_norm': 15.712736129760742, 'learning_rate': 2.9159220146222584e-05, 'epoch': 0.23}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.8040960431098938, 'eval_accuracy': 0.6770833333333334, 'eval_f1_score': 0.5467132505175983, 'eval_runtime': 7.3571, 'eval_samples_per_second': 104.39, 'eval_steps_per_second': 13.049, 'epoch': 0.23}
{'loss': 0.8409, 'grad_norm': 4.847982406616211, 'learning_rate': 2.854995938261576e-05, 'epoch': 0.29}
{'loss': 0.8059, 'grad_norm': 4.376002788543701, 'learning_rate': 2.7940698619008936e-05, 'epoch': 0.35}
{'loss': 0.8375, 'grad_norm': 4.306921482086182, 'learning_rate': 2.7331437855402112e-05, 'epoch': 0.41}
{'loss': 0.8127, 'grad_norm': 4.305049896240234, 'learning_rate': 2.672217709179529e-05, 'epoch': 0.46}


  0%|          | 0/96 [00:00<?, ?it/s]

...

  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.7006747126579285, 'eval_accuracy': 0.7330729166666666, 'eval_f1_score': 0.7338659983299061, 'eval_runtime': 7.3014, 'eval_samples_per_second': 105.185, 'eval_steps_per_second': 13.148, 'epoch': 2.55}
{'loss': 0.4886, 'grad_norm': 15.463045120239258, 'learning_rate': 4.191714053614947e-06, 'epoch': 2.6}
{'loss': 0.4008, 'grad_norm': 17.250072479248047, 'learning_rate': 3.59463850528026e-06, 'epoch': 2.66}
{'loss': 0.5307, 'grad_norm': 17.52010154724121, 'learning_rate': 2.9853777416734364e-06, 'epoch': 2.72}
{'loss': 0.3968, 'grad_norm': 4.136968612670898, 'learning_rate': 2.3761169780666128e-06, 'epoch': 2.78}


  0%|          | 0/96 [00:00<?, ?it/s]

{'eval_loss': 0.7001147270202637, 'eval_accuracy': 0.74609375, 'eval_f1_score': 0.7484941017842456, 'eval_runtime': 7.2675, 'eval_samples_per_second': 105.676, 'eval_steps_per_second': 13.21, 'epoch': 2.78}
{'loss': 0.4868, 'grad_norm': 14.654212951660156, 'learning_rate': 1.7668562144597887e-06, 'epoch': 2.84}
{'loss': 0.4416, 'grad_norm': 12.36707878112793, 'learning_rate': 1.157595450852965e-06, 'epoch': 2.89}
{'loss': 0.3789, 'grad_norm': 18.784025192260742, 'learning_rate': 5.483346872461414e-07, 'epoch': 2.95}
{'train_runtime': 758.2955, 'train_samples_per_second': 27.326, 'train_steps_per_second': 3.418, 'train_loss': 0.6213125343675967, 'epoch': 3.0}


TrainOutput(global_step=2592, training_loss=0.6213125343675967, metrics={'train_runtime': 758.2955, 'train_samples_per_second': 27.326, 'train_steps_per_second': 3.418, 'total_flos': 2744905918178304.0, 'train_loss': 0.6213125343675967, 'epoch': 3.0})

In [17]:
trainer.save_model("./results/climateBERT-base/climate_fever/first_run")
# Please remember to delete model.safetensors BEFORE adding to git. Causes issues...
# Also it is probably not worth running this block until the model is worth keeping

In [ ]:
# Metrics are not included in the save model so we need to save them separately
metrics = trainer.evaluate()
with open("./results/climateBERT-base/climate_fever/first_run/eval_metrics.json", "w") as output_file:
    json.dump(metrics, output_file)

  0%|          | 0/96 [00:00<?, ?it/s]

: 